In [1]:
from __future__ import print_function

%matplotlib inline

import sys
import tensorflow as tf
from collections import OrderedDict

import numpy as np
import scipy.misc
from input import *
from model import *
from train import *
import matplotlib.pyplot as plt

1.3.0


In [2]:
tf.reset_default_graph()

In [3]:
iterator, filename = get_train_inputs(batch_size=100,repeat=False,num_classes=2)

In [ ]:
with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={filename: ['data/kitti_segmentation.tfrecord']})
    next_element = iterator.get_next()
    while True:
        image, label = sess.run(next_element)
        plt.imshow(image)
        plt.show()
    #plt.imshow(label[:,:,0], cmap='jet', alpha=0.5)

In [ ]:
# record_iterator = tf.python_io.tf_record_iterator(path='data/kitti_segmentation.tfrecord')
# string_record = next(record_iterator)
# example = tf.train.Example()
# example.ParseFromString(string_record)

# train

In [4]:
encoder = SlimModelEncoder(name="vgg_16",num_classes=2,is_training=True)
image, label = iterator.get_next()
init_fn, end_points = encoder.build(image)

INFO:tensorflow:Scale of 0 disables regularizer.


In [ ]:
end_points.keys()

In [ ]:
end_points['vgg_16/conv5/conv5_3'], end_points['vgg_16/conv4/conv4_3'], end_points['vgg_16/conv3/conv3_3']

In [5]:
decoder = FCNDecoder(end_points, nb_classes=21, scope='fcn')

In [6]:
tensors_to_connect = OrderedDict()
tensors_to_connect['vgg_16/conv5/conv5_3'] = (2,2)
tensors_to_connect['vgg_16/conv4/conv4_3'] = (2,2)
tensors_to_connect['vgg_16/conv3/conv3_3'] = (4,4)

In [7]:
net = decoder.build(tensors_to_connect)

In [8]:
trainer = Trainer(nb_classes=2, optimizer=tf.train.AdamOptimizer, learning_rate=1e-4)

In [11]:
trainer.build(net, label)

In [ ]:
trainer.train()